# Homework 1

1. Построить список уникальных типов самолетов, зарегистрированных в России
2. Какой тип самолета имеет самую позднюю дату выдчи сертификата
3. Построить запрос: Владелец Аэропорта, Аэропорт, Пассажиропоток суммарный за 2018, грузопоток суммарный за 2018
4. Перечислить аэропорты, где пассажиропоток больше медианы, а грузопоток меньше медианы
5. Перечислить авиакомпании у которых нет типов воздушных судов зарегистрированных не в России
6. Выведите список: Месяц, суммарный пассажиропоток за данный месяц, аэропорт, в котором пассажиропоток в данном месяце минимальный
7. Вывести список: тип аэропорта, средний грузопоток в месяц в аэропортах данного типа

In [173]:
import pandas as pd
import numpy as np
aircraft = pd.read_csv("./air_stats/aircraft.csv", sep=";")
airlines = pd.read_csv("./air_stats/airlines.csv", sep=",")
airports = pd.read_csv("./air_stats/airports.csv", sep=",")
cargo = pd.read_csv("./air_stats/cargo transportation.csv", sep=";")
passenger = pd.read_csv("./air_stats/passenger transportation.csv", sep=";")

### Смотрим таблички

In [164]:
aircraft.info()
aircraft.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8933 entries, 0 to 8932
Data columns (total 7 columns):
 #   Column                                         Non-Null Count  Dtype 
---  ------                                         --------------  ----- 
 0   Вид воздушного судна                           8933 non-null   object
 1   Тип (наименование) воздушного судна            8933 non-null   object
 2   Опознавательные знаки                          8933 non-null   object
 3   Серийный №                                     5916 non-null   object
 4   Идентификац. №                                 3014 non-null   object
 5   номер свидетельства о регистрации              8933 non-null   object
 6   дата действующего свидетельства о регистрации  8933 non-null   object
dtypes: object(7)
memory usage: 488.7+ KB


,Вид воздушного судна,Тип (наименование) воздушного судна,Опознавательные знаки,Серийный №,Идентификац. №,номер свидетельства о регистрации,дата действующего свидетельства о регистрации
0,самолет,Птенец-2,RA-0001A,NaN,ЕЭВС.03.1529,0001,11.03.2012
1,дельталет,Аватар,RA-0002A,NaN,ЕЭВС.03.1709,0002,16.03.2012
2,самолет,С-2 Елочка,RA-0002G,NaN,ЕЭВС.02.0126,0224,02.06.2006
3,дельталет,СОКОЛ,RA-0003A,NaN,ЕЭВС.03.1708,0003,16.03.2012
4,самолет,Небесный,RA-0003G,NaN,ЕЭВС.02.0013,0225,07.04.2008


In [165]:
airlines.info()
airlines.head()

<class 'pandas.core.frame.DataFrame'>
Index: 116 entries, 551 to 529
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Индекс                116 non-null    object
 1   Название              116 non-null    object
 2   Юридическое Название  114 non-null    object
 3   Самолеты              114 non-null    object
dtypes: object(4)
memory usage: 4.5+ KB


,Индекс,Название,Юридическое Название,Самолеты
551,Абакан Эир,Общество с ограниченной ответственностью «АБАК...,Абакан,"Ил-76 (7), Ми-26 (3), Ми-8 (3), Ми-8МТВ (5)"
481,Авиа Менеджмент Груп,Акционерное общество «Авиа Менеджмент Груп»,Иваново (Южный),Pilatus PC-12 (10)
554,Авиакомпания «Лайт Эйр»,Закрытое акционерное общество Авиакомпания «Ла...,Уфа,"BK-117 (1), Robinson R-44 (17), Ан-2 (7), R..."
284,Авиакомпания СОКОЛ,Закрытое акционерное общество Авиакомпания «С...,Новотитаровская-Азимут,"Ми-2 (1), Ми-8 (1)"
442,Авиакон Цитотранс,Открытое акционерное общество Авиакомпания «А...,Екатеринбург(Кольцово),Ил-76 (5)


In [166]:
airports.info()
airports.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282 entries, 0 to 281
Data columns (total 4 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Наименование аэропорта РФ  281 non-null    object
 1   Свидетельство              282 non-null    object
 2   Юрлицо                     282 non-null    object
 3   Тип                        282 non-null    object
dtypes: object(4)
memory usage: 8.9+ KB


,Наименование аэропорта РФ,Свидетельство,Юрлицо,Тип
0,Архангельск (Талаги),"Сертификат МАК № 061А-М, 27.09.2013-03.08.2018...",ОАО «Аэропорт Архангельск»,В
1,"Нарьян-Мар В 3 км восточнее г.Нарьян-Мар, НАО",Свидетельство о государственной регистрации и ...,ОАО «Нарьян-Марский объединенный авиаотряд»,В
2,"Васьково 12 км юго-западнее г.Архангельск, Ар...",Свидетельство о государственной регистрации и ...,ОАО «2-ой Архангельский объединенный авиаотряд»,Г
3,"Котлас 3 км юго-восточнее г.Котлас, Архангель...",Свидетельство о государственной регистрации и ...,ОАО «Аэросервис»,Г
4,"Лешуконское 1,3 км юго-западнее села Лешуконс...",Свидетельство о государственной регистрации и ...,ОАО «Аэропорт Лешуконское»,Г


In [174]:
cargo.info()
cargo.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3473 entries, 0 to 3472
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Наименование аэропорта РФ  3473 non-null   object
 1   Год периода данных         3473 non-null   int64 
 2   Январь                     3473 non-null   object
 3   Февраль                    3473 non-null   object
 4   Март                       3473 non-null   object
 5   Апрель                     3473 non-null   object
 6   Май                        3473 non-null   object
 7   Июнь                       3473 non-null   object
 8   Июль                       3473 non-null   object
 9   Август                     3473 non-null   object
 10  Сентябрь                   3473 non-null   object
 11  Октябрь                    3212 non-null   object
 12  Ноябрь                     3212 non-null   object
 13  Декабрь                    3212 non-null   object
 14  Январь -

,Наименование аэропорта РФ,Год периода данных,Январь,Февраль,Март,Апрель,Май,Июнь,Июль,Август,Сентябрь,Октябрь,Ноябрь,Декабрь,Январь - Декабрь
0,Абакан,2018,"63,39","60,98","69,46","88,62","83,81","64,99","49,64","70,26","87,35",NaN,NaN,NaN,NaN
1,Айхал,2018,***,***,***,***,***,***,***,***,***,NaN,NaN,NaN,NaN
2,Алдан,2018,***,***,***,***,***,***,***,***,***,NaN,NaN,NaN,NaN
3,Амдерма,2018,***,***,***,***,***,***,***,***,***,NaN,NaN,NaN,NaN
4,Анадырь,2018,"112,58","267,65","221,77","331,37","160,45","89,77","109,29","95,99","103,77",NaN,NaN,NaN,NaN


In [171]:
passenger.info()
passenger.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3473 entries, 0 to 3472
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Наименование аэропорта РФ  3473 non-null   object 
 1   Год периода данных         3473 non-null   int64  
 2   Январь                     845 non-null    float64
 3   Февраль                    845 non-null    float64
 4   Март                       844 non-null    float64
 5   Апрель                     844 non-null    float64
 6   Май                        844 non-null    float64
 7   Июнь                       844 non-null    float64
 8   Июль                       848 non-null    float64
 9   Август                     861 non-null    float64
 10  Сентябрь                   861 non-null    float64
 11  Октябрь                    766 non-null    float64
 12  Ноябрь                     766 non-null    float64
 13  Декабрь                    766 non-null    float

,Наименование аэропорта РФ,Год периода данных,Январь,Февраль,Март,Апрель,Май,Июнь,Июль,Август,Сентябрь,Октябрь,Ноябрь,Декабрь,Январь - Декабрь
0,Абакан,2018,11851.0,10504.0,11591.0,12323.0,14314.0,19935.0,25213.0,25507.0,19295.0,NaN,NaN,NaN,NaN
1,Айхал,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Алдан,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Амдерма,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Анадырь,2018,5128.0,4447.0,5456.0,7650.0,8364.0,10563.0,13402.0,13751.0,10065.0,NaN,NaN,NaN,NaN


### Чистим данные

In [169]:
mounths = ['Январь', 'Февраль',
       'Март', 'Апрель', 'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь',
       'Октябрь', 'Ноябрь', 'Декабрь']
mounths_year = mounths + ['Январь - Декабрь']
for m in mounths_year:
   cargo[m].loc[cargo[m] == '***'] = np.NaN
   cargo[m] = cargo[m].apply(lambda x: float(x.replace(',', '.').replace(' ', '')) if type(x) == str else x)
   passenger[m].loc[passenger[m] == '***'] = np.NaN
   passenger[m] = passenger[m].apply(lambda x: float(x.replace(',', '.').replace(' ', '')) if type(x) == str else x)

/var/folders/tc/jj24qwg57bxbztj86x9xr0wm0000gq/T/ipykernel_64803/3434440069.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cargo[m].loc[cargo[m] == '***'] = np.NaN
/var/folders/tc/jj24qwg57bxbztj86x9xr0wm0000gq/T/ipykernel_64803/3434440069.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passenger[m].loc[passenger[m] == '***'] = np.NaN
/var/folders/tc/jj24qwg57bxbztj86x9xr0wm0000gq/T/ipykernel_64803/3434440069.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [155]:
for name in cargo['Наименование аэропорта РФ'].unique():
    data_by_city = cargo[cargo['Наименование аэропорта РФ']==name]
    data_by_city = data_by_city[mounths_year]
    if len(data_by_city.loc[data_by_city.isnull().all(1)]) == len(data_by_city):
        cargo = cargo.drop([i for i, _ in data_by_city.iterrows()])

for name in passenger['Наименование аэропорта РФ'].unique():
    data_by_city = passenger[passenger['Наименование аэропорта РФ']==name]
    data_by_city = data_by_city[mounths_year]
    if len(data_by_city.loc[data_by_city.isnull().all(1)]) == len(data_by_city):
        passenger = passenger.drop([i for i, _ in data_by_city.iterrows()])

In [156]:
for name in cargo['Наименование аэропорта РФ'].unique():
    data_by_city = cargo[cargo['Наименование аэропорта РФ']==name]
    data_by_city = data_by_city[mounths_year]
    for i, row in data_by_city.iterrows():
        if np.isnan(row['Январь - Декабрь']):
            for mounth in mounths:
                if np.isnan(row[mounth]):
                    row[mounth] = data_by_city.mean()[mounth]
            row = row.fillna(0)
            row['Январь - Декабрь'] = row[mounths].sum()
            data_by_city.loc[i] = row
    cargo.loc[cargo['Наименование аэропорта РФ']==name, mounths_year] = data_by_city

for name in passenger['Наименование аэропорта РФ'].unique():
    data_by_city = passenger[passenger['Наименование аэропорта РФ']==name]
    data_by_city = data_by_city[mounths_year]
    for i, row in data_by_city.iterrows():
        if np.isnan(row['Январь - Декабрь']):
            for mounth in mounths:
                if np.isnan(row[mounth]):
                    row[mounth] = data_by_city.mean()[mounth]
            row = row.fillna(0)
            row['Январь - Декабрь'] = row[mounths].sum()
            data_by_city.loc[i] = row
    passenger.loc[passenger['Наименование аэропорта РФ']==name, mounths_year] = data_by_city

In [146]:
cargo.info()
cargo.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1053 entries, 0 to 3471
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Наименование аэропорта РФ  1053 non-null   object 
 1   Год периода данных         1053 non-null   int64  
 2   Январь                     1053 non-null   float64
 3   Февраль                    1053 non-null   float64
 4   Март                       1053 non-null   float64
 5   Апрель                     1053 non-null   float64
 6   Май                        1053 non-null   float64
 7   Июнь                       1053 non-null   float64
 8   Июль                       1053 non-null   float64
 9   Август                     1053 non-null   float64
 10  Сентябрь                   1053 non-null   float64
 11  Октябрь                    1053 non-null   float64
 12  Ноябрь                     1053 non-null   float64
 13  Декабрь                    1053 non-null   float64
 1

,Наименование аэропорта РФ,Год периода данных,Январь,Февраль,Март,Апрель,Май,Июнь,Июль,Август,Сентябрь,Октябрь,Ноябрь,Декабрь,Январь - Декабрь
0,Абакан,2018,63.39,60.98,69.46,88.62,83.81,64.99,49.64,70.26,87.35,87.388182,91.773636,113.661818,931.323636
4,Анадырь,2018,112.58,267.65,221.77,331.37,160.45,89.77,109.29,95.99,103.77,159.211818,169.090909,288.170000,2109.112727
5,Анапа(Витязево),2018,43.38,50.43,50.43,61.64,63.94,77.29,82.30,81.20,75.00,46.580909,41.993636,49.596364,723.780909
8,Архангельск(Талаги),2018,119.56,130.61,166.54,211.94,214.98,132.86,122.48,165.11,121.52,235.210000,342.932727,514.773636,2478.516364
9,Астрахань(Нариманово),2018,47.67,53.47,63.11,69.69,66.51,53.62,68.75,77.58,76.00,72.547273,64.610909,82.157273,795.715455


In [157]:
passenger.info()
for i, row in passenger.iterrows():
    print(row)

<class 'pandas.core.frame.DataFrame'>
Index: 1053 entries, 0 to 3471
Data columns (total 15 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Наименование аэропорта РФ  1053 non-null   object 
 1   Год периода данных         1053 non-null   int64  
 2   Январь                     1053 non-null   float64
 3   Февраль                    1053 non-null   float64
 4   Март                       1053 non-null   float64
 5   Апрель                     1053 non-null   float64
 6   Май                        1053 non-null   float64
 7   Июнь                       1053 non-null   float64
 8   Июль                       1053 non-null   float64
 9   Август                     1053 non-null   float64
 10  Сентябрь                   1053 non-null   float64
 11  Октябрь                    1053 non-null   float64
 12  Ноябрь                     1053 non-null   float64
 13  Декабрь                    1053 non-null   float64
 1

### Cписок уникальных типов самолетов, зарегистрированных в России

In [149]:
print(*aircraft.loc[aircraft["Вид воздушного судна"] == "самолет"]["Тип (наименование) воздушного судна"].unique().tolist(),sep='\n')

Птенец-2
С-2 Елочка
Небесный
Светлый
Сура
EUROSTAR
Pioneer 330
СП-30З О
Саргон
Аэропракт-20
Ястреб
Рейнджер МПВ
Икарус С42В
С22В Икарус
Авиатика-890СХ
С-2УТ Синтал
Х-32-912УТ Гор
Peregrrine SL
EuroStar SL
EuroStar SLW
СП-34М
Саванна
FK-9 Mark-3
Z-602
А-27МС
Дельфин-10
АС-5М Альбатрос
Вимана
Аэропракт-22L2
Авиатика-МАИ-890СХ
Жучок
СП-30 Эльф
А-27М
СП-30
Спорткруизер
СН-701
СП-30Амур1
NG-46
ЭКЛИПС
Alpha Trainer
Амик-10
Sky Ranger
Х320
Аэропракт-22Л2
ТОПОЛЬ
МК 192
СП-01(КП)
Р-20 "Птенец-2"
СТ-18
СКИФ
САПСАН
WT9 DS
Петр Первый
Авиатика-МАИ-890У
ОБ-31
Ан-2
Ан-2Т
СП-30 Ермак
Викинг
X-32 B-1 "Бекас"
Z-43
А-27М2
СК-12т3 "Орион"
KP-2U SOVA
ФОТОН
Касатик М-12К
Savannah
Корвет-26
Дельфин_
По-2
Таурус М
СТ-2000
Sky Ranger G
СП-30НГ
Ан-2ТП
СП-30К
Корвет-Л
СП-30Z
ТВС-2АМ-1
БЕРКУТ-1
NG-5BRM
ТВС-2МС
BLACKSHAPE CF300 "PRIME"
П2002 Сиерра
С-2СХ"Синтал"
PC-12/47
PC-12/47E
PC-12/45
Авиатика 890-582
Pilatus PC-12/45
PC-6/B2-H4
Pilatus PC-12/47E
Аист-410М
P.180
Piaggio FW P 149D
П92 Эхо Супер
СП-30бви
Х-32 

### Какой тип самолета имеет самую позднюю дату выдчи сертификата

Переводим дату к нужному типу данных

In [150]:
aircraft["дата действующего свидетельства о регистрации"] = pd.to_datetime(aircraft["дата действующего свидетельства о регистрации"], format="%d.%m.%Y")

In [151]:
print(*aircraft.loc[
    (aircraft["дата действующего свидетельства о регистрации"] == aircraft["дата действующего свидетельства о регистрации"].max()) & 
    (aircraft["Вид воздушного судна"] == "самолет")]["Тип (наименование) воздушного судна"].values)

E 330 LXK


### Построить запрос: Владелец Аэропорта, Аэропорт, Пассажиропоток суммарный за 2018, грузопоток суммарный за 2018

Чистим и приводин к числовым типам данные

In [21]:
for m in ['Октябрь', 'Ноябрь', 'Декабрь']:
   cargo.loc[cargo['Год периода данных'] , m] = 0
for m in ['Январь', 'Февраль',
       'Март', 'Апрель', 'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь',
       'Октябрь', 'Ноябрь', 'Декабрь']:
   # cargo[m] = cargo[m].fillna(0)
   cargo[m].loc[cargo[m] == '***'] = np.NaN
   cargo[m] = cargo[m].apply(lambda x: float(x.replace(',', '.').replace(' ', '')) if type(x) == str else x)
#    passenger[m] = passenger[m].fillna(0)
   passenger[m].loc[passenger[m] == '***'] = np.NaN
   passenger[m] = passenger[m].apply(lambda x: float(x.replace(',', '.').replace(' ', '')) if type(x) == str else x)

/var/folders/tc/jj24qwg57bxbztj86x9xr0wm0000gq/T/ipykernel_64803/915010144.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cargo[m].loc[cargo[m] == '***'] = np.NaN
/var/folders/tc/jj24qwg57bxbztj86x9xr0wm0000gq/T/ipykernel_64803/915010144.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passenger[m].loc[passenger[m] == '***'] = np.NaN
/var/folders/tc/jj24qwg57bxbztj86x9xr0wm0000gq/T/ipykernel_64803/915010144.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#

In [155]:
cargo['Грузопоток за год'] = cargo[['Январь', 'Февраль',
       'Март', 'Апрель', 'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь',
       'Октябрь', 'Ноябрь', 'Декабрь']].sum(axis=1)
passenger['Пасажиропоток за год'] = passenger[['Январь', 'Февраль',
       'Март', 'Апрель', 'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь',
       'Октябрь', 'Ноябрь', 'Декабрь']].sum(axis=1)

In [159]:
company_year_sum = pd.merge(
    pd.merge(
        airports[['Наименование аэропорта РФ', 'Юрлицо']], 
        passenger[['Наименование аэропорта РФ', 'Год периода данных', 'Пасажиропоток за год']]),  
    cargo[['Наименование аэропорта РФ', 'Год периода данных', 'Грузопоток за год']], 
    left_on=('Наименование аэропорта РФ', 'Год периода данных'), right_on=('Наименование аэропорта РФ', 'Год периода данных')
)
company_year_sum.loc[company_year_sum['Год периода данных']==2018]

,Наименование аэропорта РФ,Юрлицо,Год периода данных,Пасажиропоток за год,Грузопоток за год
0,Ухта,ОАО «Комиавиатранс»,2018,0.0,0.00
12,Усинск,ОАО «Комиавиатранс»,2018,0.0,0.00
24,Печора,ОАО «Комиавиатранс»,2018,0.0,0.00
36,Братск,ПАО «АэроБратск»,2018,0.0,0.00
48,Усть-Кут,ОАО «Аэропорт Усть-Кут»,2018,0.0,0.00
60,Мама,ООО «Мамский аэропорт»,2018,0.0,0.00
72,Бодайбо,"Бодайбинское авиапредприятие, ЗАО «Ленсиб»",2018,0.0,0.00
84,Киренск,ООО «Аэропорт «Киренск»,2018,0.0,0.00
96,Ербогачен,ООО «Аэропорт «Киренск»,2018,0.0,0.00
108,Нижнеудинск,ООО «Аэропорт «Нижнеудинск»,2018,0.0,0.00


In [172]:
cargo[['Январь', 'Февраль',
       'Март', 'Апрель', 'Май', 'Июнь', 'Июль', 'Август', 'Сентябрь',
       'Октябрь', 'Ноябрь', 'Декабрь']].median()

Январь      109.490
Февраль     148.860
Март        172.050
Апрель      177.050
Май         161.180
Июнь        149.335
Июль        141.770
Август      149.400
Сентябрь    154.700
Октябрь     184.530
Ноябрь      183.225
Декабрь     231.610
dtype: float64

In [181]:
cargo.loc[cargo['Наименование аэропорта РФ'] == 'Абакан']

,Наименование аэропорта РФ,Год периода данных,Январь,Февраль,Март,Апрель,Май,Июнь,Июль,Август,Сентябрь,Октябрь,Ноябрь,Декабрь,Январь - Декабрь
0,Абакан,2018,63.39,60.98,69.46,88.62,83.81,64.99,49.64,70.26,87.35,0.00,0.00,0.00,NaN
261,Абакан,2017,48.36,77.08,95.33,90.28,91.93,57.13,82.59,81.20,76.05,139.67,71.58,88.84,"1000,04"
553,Абакан,2016,34.10,45.41,58.97,72.71,91.66,78.49,64.31,127.29,89.07,74.99,78.66,94.01,"909,67"
845,Абакан,2015,37.70,47.97,54.67,82.12,68.81,112.95,55.83,95.20,137.79,72.13,63.67,78.30,"907,14"
1137,Абакан,2014,51.90,69.42,83.36,69.71,75.50,92.16,67.50,89.22,76.95,78.56,153.68,83.38,"991,34"
1429,Абакан,2013,39.74,227.16,62.97,77.41,104.34,55.71,65.95,78.71,75.68,70.30,81.57,83.34,"1022,88"
1721,Абакан,2012,39.24,39.24,60.58,55.53,56.51,57.35,64.47,75.88,66.40,120.62,64.38,81.25,"781,45"
2013,Абакан,2011,53.30,59.40,162.40,72.40,90.10,92.10,81.30,73.20,77.00,84.30,91.17,124.56,"1061,23"
2305,Абакан,2010,35.10,192.20,55.20,108.70,118.80,69.30,87.70,77.00,181.40,88.50,83.30,110.60,"1207,8"
2597,Абакан,2009,40.10,36.90,50.30,47.00,58.20,77.60,83.90,404.70,65.10,68.20,182.20,362.50,"1476,7"


In [36]:
for i, row in cargo.loc[[261]].iterrows():
    print(row[["Январь", "Февраль",'Март','Апрель','Май','Июнь','Июль','Август','Сентябрь','Октябрь','Ноябрь','Декабрь']].sum())

1000.04


In [37]:
cargo.loc[[261]]

,Наименование аэропорта РФ,Год периода данных,Январь,Февраль,Март,Апрель,Май,Июнь,Июль,Август,Сентябрь,Октябрь,Ноябрь,Декабрь,Январь - Декабрь
261,Абакан,2017,48.36,77.08,95.33,90.28,91.93,57.13,82.59,81.2,76.05,139.67,71.58,88.84,1000.04


In [152]:
passenger.loc[cargo['Наименование аэропорта РФ'] == 'Ростов-на-Дону']

,Наименование аэропорта РФ,Год периода данных,Январь,Февраль,Март,Апрель,Май,Июнь,Июль,Август,Сентябрь,Октябрь,Ноябрь,Декабрь,Январь - Декабрь
164,Ростов-на-Дону,2018,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN
450,Ростов-на-Дону,2017,188648,170579,193315,197326,232182,247027,270007,264191,250469,242538,233072,42912,2532266
742,Ростов-на-Дону,2016,136594,126220,136329,145699,172718,187573,205961,211190,203029,187431,186745,193771,2093260
1034,Ростов-на-Дону,2015,142741,122433,140539,137852,161747,203884,240651,248101,207358,172475,139961,143274,2061016
1326,Ростов-на-Дону,2014,144216,127141,158647,179511,229351,254147,276528,273767,194749,180543,161803,159892,2340295
1618,Ростов-на-Дону,2013,135503,124682,144233,159855,195657,215646,234159,252337,225707,185044,160874,155742,2189439
1910,Ростов-на-Дону,2012,121110,108184,118669,133839,163138,182444,196343,208877,190790,158846,148439,138236,1868915
2202,Ростов-на-Дону,2011,103846,80394,98024,117168,150685,181516,200391,201148,179502,145355,130755,127294,1716078
2494,Ростов-на-Дону,2010,81118,74096,90725,102565,132007,138845,159653,164389,149856,132640,108320,104409,1438623
2786,Ростов-на-Дону,2009,67756,59412,67022,76930,92438,109153,123831,141563,119825,105278,85797,88771,1137776
